In [68]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import category_encoders as ce
import statsmodels.formula.api as smf
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from xgboost import train_values_copyGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer


train_values = pd.read_csv('train_values.csv', sep=',')
train_labels = pd.read_csv('train_labels.csv', sep=',')
x_test = pd.read_csv('test_values.csv', sep=',')

train_values_copy = train_values.copy()
train_values_copy.drop('building_id', inplace=True, axis=1)
y = train_labels.copy()
y.drop('building_id', inplace=True, axis=1)
x_test_dropped = x_test.copy()
x_test_dropped.drop('building_id', inplace=True, axis=1)


     |████████████████████████████████| 1.7 MB 2.3 MB/s eta 0:00:01


In [69]:
train_values_copy[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]] = train_values_copy[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]].astype(str)
x_test_dropped[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]] = x_test_dropped[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]].astype(str)


In [70]:
#generate the encoded columns
first_ce = ce.OneHotEncoder(cols=["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","legal_ownership_status"]) 
ce_te = ce.TargetEncoder(cols=["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"])
xx = train_values_copy[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]]
x_test_target_encoded = x_test_dropped[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]]
yy = y['damage_grade']

#fit and transform the target encoded values
ce_te.fit(xx,yy)
train_values_copy[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]] = ce_te.transform(xx)
x_test_dropped[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]] = ce_te.transform(x_test_target_encoded)

#fit and transform the one hot encoded values
train_values_copy = first_ce.fit_transform(train_values_copy)
x_test_dropped = first_ce.fit_transform(x_test_dropped)

In [ ]:
reg = LinearRegression().fit(train_values_copy, y)
    
est = sm.OLS(y, train_values_copy)
est2 = est.fit()
est2.summary()

In [63]:
dropped_columns_encoded_x_train = train_values_copy.drop(columns=["count_floors_pre_eq","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"])
dropped_columns_encoded_x_test = x_test_dropped.drop(columns=["count_floors_pre_eq","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"])

In [67]:
from sklearn.ensemble import RandomForestClassifier

reg = LinearRegression().fit(dropped_columns_encoded_x_train, y)
    
est = sm.OLS(y, dropped_columns_encoded_x_train)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           damage_grade   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.468
Method:                 Least Squares   F-statistic:                     6193.
Date:                Fri, 29 Jul 2022   Prob (F-statistic):               0.00
Time:                        14:04:12   Log-Likelihood:            -1.5953e+05
No. Observations:              260601   AIC:                         3.191e+05
Df Residuals:                  260563   BIC:                         3.195e+05
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
geo_level_1_id                             0.0406      0.005      7.684      0.000       0.030       0.051
geo_level_2_id                            -0.0379      0.007     -5.819      0.000      -0.051      -0.025
geo_level_3_id                             0.9133      0.005    179.908      0.000       0.903       0.923
age                                        0.0001    1.2e-05      9.960      0.000    9.62e-05       0.000
area_percentage                           -0.0004      0.000     -1.783      0.075      -0.001    3.93e-05
height_percentage                          0.0026      0.001      4.274      0.000       0.001       0.004
land_surface_condition_1                  -0.1032      0.010    -10.411      0.000      -0.123      -0.084
land_surface_condition_2                  -0.0927      0.010     -8.855      0.000      -0.113      -0.072
land_surface_condition_3                  -0.1113      0.010    -11.104      0.000      -0.131      -0.092
foundation_type_1                          0.0255      0.007      3.720      0.000       0.012       0.039
foundation_type_2                         -0.0316      0.007     -4.397      0.000      -0.046      -0.018
foundation_type_3                         -0.2297      0.009    -25.311      0.000      -0.248      -0.212
foundation_type_4                         -0.0355      0.007     -4.872      0.000      -0.050      -0.021
foundation_type_5                         -0.0358      0.012     -3.089      0.002      -0.059      -0.013
roof_type_1                               -0.0624      0.010     -6.251      0.000      -0.082      -0.043
roof_type_2                               -0.0472      0.010     -4.700      0.000      -0.067      -0.028
roof_type_3                               -0.1975      0.011    -18.103      0.000      -0.219      -0.176
ground_floor_type_1                        0.0056      0.008      0.750      0.453      -0.009       0.020
ground_floor_type_2                       -0.0118      0.008     -1.528      0.126      -0.027       0.003
ground_floor_type_3                       -0.0907      0.008    -11.343      0.000      -0.106      -0.075
ground_floor_type_4                       -0.1012      0.014     -7.411      0.000      -0.128      -0.074
ground_floor_type_5                       -0.1090      0.018     -6.210      0.000      -0.143      -0.075
other_floor_type_1                        -0.0792      0.008    -10.315      0.000      -0.094      -0.064
other_floor_type_2                        -0.0733      0.008     -9.438      0.000      -0.088      -0.058
other_floor_type_3                        -0.0650      0.008     -8.484      0.000      -0.080      -0.050
other_floor_type_4                        -0.0897      0.009    -10.080      0.000     

In [64]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(min_samples_split=20, n_estimators=200)
clf = clf.fit(dropped_columns_encoded_x_train,y)
y_pred=clf.predict(dropped_columns_encoded_x_test)
y_pred

array([3, 2, 3, ..., 2, 2, 2])

In [10]:
from sklearn.model_selection import cross_val_score
y_train_vector = y.values.flatten()
print(np.mean(cross_val_score(clf, dropped_columns_encoded_x_train, y_train_vector)))

0.7408145070896852


In [59]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.2, random_state=0).fit(dropped_columns_encoded_x_train, y)
ypred_gradient_boosting = clf.predict(dropped_columns_encoded_x_test)
ypred_gradient_boosting


array([3, 2, 3, ..., 2, 2, 2])

In [55]:

#iris = datasets.load_iris()
estimator = train_values_copyGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)
parameters = {
    'max_depth': [6, 10],
    'learning_rate': [0.1, 0.2]
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    verbose=True,
    error_score='raise'
)
grid_search.fit(dropped_columns_encoded_x_train, y.values.flatten() - 1)

sorted(grid_search.cv_results_.keys())
grid_search.best_estimator_


Fitting 5 folds for each of 4 candidates, totalling 20 fits


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=42, ...)

In [73]:
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
xgbc = XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              monotone_constraints='()', n_estimators=100,
              n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=42,)
print(xgbc)
xgbc.fit(dropped_columns_encoded_x_train, y-1)
ypred_xgboost = xgbc.predict(dropped_columns_encoded_x_test) + 1

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=42, ...)


In [31]:
ypred_xgboost

array([3, 2, 3, ..., 2, 2, 2])

In [65]:
from pathlib import Path  
submission_random_forest = x_test["building_id"]
i = pd.DataFrame(np.array(y_pred))

submission_concatenated = pd.concat([submission_random_forest, i], ignore_index="true", axis=1)
submission_renamed_columns = submission_concatenated.rename(columns={0: "building_id", 1: "damage_grade"})
filepath = Path('./submission_richter_prediction.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
submission_renamed_columns.to_csv(filepath, index=False)